In [ ]:
!pip install transformers

In [ ]:
!nvidia-smi

Sat Jun  1 15:00:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              13W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
#/content/drive/MyDrive/TFG/dataset2train.csv

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14677582483226467413
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14626652160
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14033888237188119344
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
!cat /proc/meminfo

MemTotal:       13290464 kB
MemFree:         7730232 kB
MemAvailable:   11950896 kB
Buffers:          352192 kB
Cached:          4057708 kB
SwapCached:            0 kB
Active:           696584 kB
Inactive:        4449176 kB
Active(anon):       1140 kB
Inactive(anon):   746412 kB
Active(file):     695444 kB
Inactive(file):  3702764 kB
Unevictable:           4 kB
Mlocked:               4 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:             34648 kB
Writeback:            92 kB
AnonPages:        735660 kB
Mapped:           741748 kB
Shmem:             11624 kB
KReclaimable:     126788 kB
Slab:             174596 kB
SReclaimable:     126788 kB
SUnreclaim:        47808 kB
KernelStack:        4976 kB
PageTables:         9768 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6645232 kB
Committed_AS:    2627764 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       79360 kB
VmallocChunk:    

In [ ]:
import torch
if torch.cuda.is_available():

  device = torch.device("cuda")
  print(torch.cuda.device_count())
  print(torch.cuda.get_device_name(0))

else:
  device = torch.device("cpu")


1
Tesla T4


In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
#import pytorch_lightning as pl
#from pytorch_lightning.metrics.functional.classification import auroc
#from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
txt_path = '/content/drive/MyDrive/TFG/dataset4ft.csv'
#txt_path = '/content/drive/MyDrive/TFG/dataset2ft.csv'
data_pd = pd.read_csv( txt_path, sep=",",encoding='cp1252')
data_pd = data_pd.rename(columns = { 'Abb': 'Abbreviation', 'Exp': 'Expansion', 2 :'ABB_frm' , "start_pos" : "start_pos", "end_pos": "end_pos", 5 : "info", 'text3': "context"}, inplace = False)
data_pd = data_pd[['Abbreviation', 'Expansion', 'start_pos', 'end_pos', 'context']]
data_pd.count()
#data_pd.info()

Abbreviation    6575
Expansion       6575
start_pos       6575
end_pos         6575
context         6575
dtype: int64

In [ ]:
data_pd1 = data_pd[data_pd['Abbreviation'] == 'AB']
print(data_pd1.shape)

(0, 5)


In [ ]:
#check number of expansion in the dataset before filtering
print(len(data_pd["Expansion"].unique()))

#filtering dataset
data_pd.drop(data_pd[data_pd['Expansion'] == "UNSURED SENSE"].index, inplace = True)
data_pd.drop(data_pd[data_pd['Expansion'] == "GENERAL ENGLISH"].index, inplace = True)
data_pd.drop(data_pd[data_pd['Expansion'] == "NAME"].index, inplace = True)

# check dataset size after filtering
print(data_pd.shape)

370
(6575, 5)


In [ ]:
# fuction to assign numerical value to the expansion
def func(unique_expansion, ex):
  ex= str(ex)
  for i in unique_expansion.items():
    if i[0] == ex:
      return i[1]


In [ ]:
Expansion_unique = data_pd["Expansion"].unique()
context = data_pd["context"].values

num = np.arange(0, len(data_pd["Expansion"].unique()), 1).tolist()
unique_expansion = dict(zip(Expansion_unique,num))

all_label = []

for index, row in data_pd.iterrows():
  for i in unique_expansion.items():
    if i[0] == row[1]:
      all_label.append(i[1])

In [ ]:
word2index_dict = {word: i for (i, word) in enumerate(Expansion_unique)}

In [ ]:
label_df = pd.DataFrame(list(word2index_dict.items()),columns = ['expansion','label'])
label_df.to_csv('/content/drive/MyDrive/TFG/label_expansion.tsv', sep = '|')
label_df['label']

0        0
1        1
2        2
3        3
4        4
      ... 
365    365
366    366
367    367
368    368
369    369
Name: label, Length: 370, dtype: int64

In [ ]:
# function to clean the text
import re
import string
def cleaning(context_):
  x = re.sub("_%#\S+", "", context_)
  x = re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
  x = re.sub(r'\w*\d+\w*','', x)
  x = re.sub('\s{2,}', " ", x)
  return x


In [ ]:
#filtered dataset after cleaning text and labeled expansion
filtered_data = pd.DataFrame()
filtered_data['content'] = data_pd['context'].apply(cleaning)
filtered_data['expansion'] = data_pd['Expansion']
filtered_data['label'] = all_label

In [ ]:
#filtered_data['label'][5000]

In [ ]:
class DisambiguateDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer, max_token_len ):

    self.data = data
    self.tokenizer = tokenizer
    self.max_token_len = max_token_len

  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, index: int):

    data_row = self.data.iloc[index]
    context =data_row['content']
    expansion = data_row['expansion']
    label_ = data_row['label']

    encoding = self.tokenizer.encode_plus(
        expansion, context,
        #text_pair = expansion,
        add_special_tokens = True,
        max_length = 512,
        return_token_type_ids = True,
        padding = "max_length",
        truncation =True,
        return_attention_mask = True,
        return_tensors = "pt"

    )
    return dict(
      input_ids = encoding["input_ids"].flatten(),
      attention_mask = encoding["attention_mask"].flatten(),
      token_type_ids = encoding["token_type_ids"].flatten(),
      label_ = torch.tensor(label_,dtype = torch.long)
    )



In [ ]:
RANDOM_SEED = 42
MAX_LEN = 512
BATCH_SIZE = 8
EPOCHS = 5

# divide the data set to training and validation dataset and check the ne w size for both
train_df, test_df = train_test_split(filtered_data,test_size=0.2, random_state = RANDOM_SEED)
val_df, test_df = train_test_split(test_df,test_size=0.5, random_state = RANDOM_SEED)
train_df.shape, val_df.shape, test_df.shape


((5260, 3), (657, 3), (658, 3))

In [ ]:
test_df.columns

Index(['content', 'expansion', 'label'], dtype='object')

In [ ]:
filtered_data.head(5)

,content,expansion,label
0,Actualmente los pacientes con cancer de pulmon...,receptor del factor de crecimiento epidermico,0
1,OBJETIVO Valorar las actividades sanitarias en...,presion arterial,1
2,INTRODUCCION Valorar modificacion en pacientes...,lipoproteinas de baja densidad,2
3,Presentamos un caso de artritis reactiva secun...,antiinflamatorios no esteroideos,3
4,Estudio de dos pacientes con dolor intenso de ...,resonancia magnetica nuclear,4


In [ ]:
# Download Bio_clinicalBERT

#from transformers import AutoTokenizer, AutoModel, BertForNextSentencePrediction, BertTokenizer
#clinic_tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
#clinic_model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to(device)

In [ ]:
#Download ms_bert

from transformers import AutoTokenizer, AutoModel
model_name = "NLP4H/ms_bert"
model = AutoModel.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 199 different named parameters.

==== Embedding Layer ====

embeddings.word_embeddings.weight                       (30522, 768)
embeddings.position_embeddings.weight                     (512, 768)
embeddings.token_type_embeddings.weight                     (2, 768)
embeddings.LayerNorm.weight                                   (768,)
embeddings.LayerNorm.bias                                     (768,)

==== First Transformer ====

encoder.layer.0.attention.self.query.weight               (768, 768)
encoder.layer.0.attention.self.query.bias                     (768,)
encoder.layer.0.attention.self.key.weight                 (768, 768)
encoder.layer.0.attention.self.key.bias                       (768,)
encoder.layer.0.attention.self.value.weight               (768, 768)
encoder.layer.0.attention.self.value.bias                     (768,)
encoder.layer.0.attention.output.dense.weight             (768, 768)
encoder.layer.0.attention.output.dense.bias                   (

In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
  ds = DisambiguateDataset(df, tokenizer, max_len)

  return torch.utils.data.DataLoader(ds, batch_size = batch_size, shuffle = True )

In [ ]:
train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:

data = next(iter(val_data_loader))
len(data)

4

In [ ]:
out = model(data['input_ids'].to(device), data['attention_mask'].to(device), data['token_type_ids'].to(device))
out[0]

tensor([[[ 1.3516e+00,  7.5663e-01, -8.2737e-03,  ...,  2.3927e-01,
          -1.0904e+00,  1.1828e-01],
         [-3.5290e-01,  4.9210e-01, -8.9272e-01,  ..., -1.2407e+00,
           9.1321e-01,  1.0725e+00],
         [-8.4773e-02, -1.2006e-01, -6.3792e-01,  ..., -1.0932e+00,
           8.5743e-02,  1.7280e+00],
         ...,
         [-1.7716e+00,  3.3820e-01, -6.4916e-01,  ..., -4.9693e-01,
           5.8935e-01,  5.4570e-02],
         [-6.8405e-01, -1.5510e+00, -1.7591e+00,  ..., -8.9243e-01,
           7.0647e-01, -2.3387e-01],
         [ 2.3100e-01, -5.8531e-01, -7.6070e-01,  ...,  2.9565e-01,
          -8.4027e-02, -2.9657e-01]],

        [[ 1.1343e+00,  1.5925e+00, -1.2882e-01,  ..., -9.0164e-01,
          -8.1000e-01,  5.6222e-01],
         [ 5.2872e-01,  1.6379e-01,  1.2291e+00,  ..., -1.8183e+00,
           2.5116e-01,  6.9217e-01],
         [-1.4565e+00, -1.9108e+00,  6.5955e-01,  ..., -9.0527e-01,
          -1.0625e+00, -2.3599e-01],
         ...,
         [ 1.1071e+00, -7

In [ ]:
correct_predictions = 0
_, preds = torch.max(out[0], dim=1)
preds[0] , data['label_']
#loss = loss_fn(out,data['label_'].to(device))
#loss
#correct_predictions += torch.sum(preds == data['label_'].to(device))
#correct_predictions
#preds[0], data['label_'].shape

(tensor([291,   7, 176, 142, 255, 424, 146, 337, 320, 398, 289, 172, 166, 179,
          52, 231,  80, 503, 317, 294, 230, 185,  85, 192, 495, 469, 234, 237,
         263,  71,  14, 509, 209,  75, 206, 381, 224, 106, 132, 207, 398, 490,
         236, 477,  48,  16, 296,  81,  55, 269,  17, 367, 323, 174, 133, 172,
         110, 506,  42,  37, 499, 112, 325, 200, 368, 145, 394,  65, 105, 110,
         353,  48, 245, 382, 311, 101, 130, 364, 166,  20, 218, 277, 101, 109,
          80, 477, 104, 407, 269,  71, 251, 164,  77, 417, 277, 105, 159,  53,
         294, 452, 462,  77, 229, 250, 208, 209, 114, 120, 290, 320,  42, 123,
         336, 314,  47, 405, 320, 100, 192,   4, 321, 481, 321, 179, 113, 273,
           2, 347, 325, 250, 339, 294, 311, 360, 233, 174,   3,  63, 433, 100,
         166, 475, 271, 327, 386, 108, 338, 488, 183, 321, 249,  24, 112,  10,
         322, 402, 252, 227, 302,  48, 164, 148, 508, 396, 394, 450, 464, 289,
         258, 335, 387,  89, 145,  53, 448, 510, 441

In [ ]:
#out['pooler_output'].shape , out['hidden_states'][0].shape

In [ ]:
class DisambiguateClassifier(nn.Module):

  def __init__(self, n_classes):
    super(DisambiguateClassifier, self).__init__()
    self.model = model
    self.linear_relu_stack = nn.Sequential(
     nn.Linear(self.model.config.hidden_size, 512),
     nn.ReLU(),
     nn.Linear(512, 370),
    )

  def forward(self, input_ids, token_type_ids, attention_mask ):
    output = self.model(
        input_ids,
       token_type_ids,
       attention_mask,
    )
    out = self.linear_relu_stack (output['last_hidden_state'][:,0,:])
    return out



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#clinic_model = DisambiguateClassifier(model)
#clinic_model.to(device)
model = DisambiguateClassifier(model)
model.to(device)

DisambiguateClassifier(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

In [ ]:
from transformers import AdamW
from transformers.optimization import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr = 1e-5 )
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
 model = model.train()

 losses = []
 correct_predictions = 0
 all_predictions = []
 all_targets = []

 for d in data_loader:
   input_ids = d['input_ids'].to(device)
   attention_mask = d['attention_mask'].to(device)
   token_type_ids = d['token_type_ids'].to(device)
   label = d['label_'].to(device)

   outputs = model(input_ids, attention_mask, token_type_ids,)

   _, preds = torch.max(outputs, dim=1)
   loss = loss_fn(outputs,label)
   correct_predictions += torch.sum(preds == label)
   losses.append(loss.item())
   loss.backward()
   nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
   optimizer.step()
   scheduler.step()
   optimizer.zero_grad()
   all_predictions.extend(preds.tolist())
   all_targets.extend(label.tolist())

 accuracy = correct_predictions.double() / n_examples
 average_loss = np.mean(losses)

 precision = precision_score(all_targets, all_predictions, average='macro')
 recall = recall_score(all_targets, all_predictions, average='macro')
 f1 = f1_score(all_targets, all_predictions, average='macro')

 return accuracy, average_loss, precision, recall, f1

In [ ]:
def eval_model(model, data_loader, loss_fn, device, n_examples):

  model = model.eval()
  losses = []
  correct_predictions = 0
  all_predictions = []
  all_targets = []
  with torch.no_grad():

    for d in data_loader:
      input_ids = d['input_ids'].to(device)
      attention_mask = d['attention_mask'].to(device)
      token_type_ids = d['token_type_ids'].to(device)
      label = d['label_'].to(device)

      outputs = model(
      input_ids,
      attention_mask,
      token_type_ids,
      )

      _,preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, label)
      for p, x, ii in zip(preds,label,input_ids):
        if p.item() != x.item():
          #print('Bad prediction')
          #print(preds)
          #print(label)
          print(p.item(),",", x.item(),",",tokenizer.decode(ii,skip_special_tokens= True))

      correct_predictions += torch.sum(preds == label)
      losses.append(loss.item())
      #prediction = torch.stack(prediction).cpu()
      all_predictions.extend(preds.tolist())
      all_targets.extend(label.tolist())

  accuracy = correct_predictions.double() / n_examples
  average_loss = np.mean(losses)

  precision = precision_score(all_targets, all_predictions, average='macro')
  recall = recall_score(all_targets, all_predictions, average='macro')
  f1 = f1_score(all_targets, all_predictions, average='macro')


  return accuracy, average_loss, precision, recall, f1

In [ ]:
EPOCHS = 5

from collections import defaultdict
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
history = defaultdict(list)

best_accuracy = 0

for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')

  print('-' * 10)

  train_acc, train_loss, train_prec, train_recall, train_f1 = train_epoch(
    model,
    train_data_loader,
    loss_fn,
    optimizer,
    device,
    scheduler,
    len(train_df)
 )

  print(f'Train loss {train_loss} accuracy {train_acc} Precision {train_prec}, Recall {train_recall}, F1 {train_f1}')

  val_acc, val_loss, val_precision, val_recall, val_f1, = eval_model(
    model,
    val_data_loader,
    loss_fn,
    device,
    len(val_df)
  )

  print(f'Val   loss {val_loss} accuracy {val_acc}')

  print(f'Val loss {val_loss} accuracy {val_acc}, Precision {val_precision}, Recall {val_recall}, F1 {val_f1}')
  print()

  history['train_acc'].append(train_acc)
  history['train_loss'].append(train_loss)
  history['val_acc'].append(val_acc)
  history['val_loss'].append(val_loss)
  history['val_precision'].append(val_precision)
  history['val_recall'].append(val_recall)
  history['val_f1'].append(val_f1)

  if val_acc > best_accuracy:
    torch.save(model.state_dict(), 'best_model_state.bin')
    best_accuracy = val_acc

Epoch 1/5
----------


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train loss 3.5213002832464895 accuracy 0.5085551330798479 Precision 0.16438681939570163, Recall 0.09663286369022317, F1 0.10745520875252239
60 , 35 , presion arterial sistolica objetivo verificar el significado clinico y el valor pronostico de la presion de pulso en poblacion general anciana diseno estudio descriptivo transversal realizado entre junio y octubre de en el ambito de atencion primaria participantes de la poblacion de edad anos n de cupos de seis centros de salud de asturias se obtuvo una muestra aleatoria simple de personas mediciones principales se analizaron variables demograficas y factores de riesgo cardiovascular y se investigo la presencia de patologia cardiovascular asociada el analisis se realizo mediante la division en terciles de la presion de pulso resultados se incluyo a personas de perdidas con una edad media de anos el eran mujeres las medias de la pas presion arterial sistolica y diastolica pad fueron y mmhg respectivamente los terciles de presion de pulso f

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/5
----------


...

Val   loss 0.6263956431389214 accuracy 0.9269406392694064
Val loss 0.6263956431389214 accuracy 0.9269406392694064, Precision 0.7291697772388959, Recall 0.7710843373493976, F1 0.7447692110488298

Epoch 5/5
----------


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train loss 0.6003414107073314 accuracy 0.9252851711026615 Precision 0.45960776585358876, Recall 0.46268515324711285, F1 0.45467459865657345
22 , 136 , inhibidor de la bomba de protones objetivo evaluar desde la perspectiva del sistema nacional de salud la eficiencia de la combinacion a dosis fija de naproxeno y esomeprazol naproxeno esomeprazol en artrosis frente a otros aine en monoterapia o combinados con un ibp inhibidor de la bomba de protones metodos se empleo un modelo de markov con estados de salud definidos por episodios gastrointestinales gi dispepsia ulcera peptica sintomatica o complicada o cardiovasculares cv infarto agudo de miocardio ictus o insuficiencia cardiaca el modelo es semejante al utilizado por el nice en su evaluacion de aine en artrosis publicada en se estimaron en un horizonte temporal de ano ciclos de meses los costes totales d incluyendo coste farmacologico y de manejo de episodios y los resultados en salud expresados en anos de vidaajustados por calidad ava

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Val   loss 0.5715601994061327 accuracy 0.9330289193302892
Val loss 0.5715601994061327 accuracy 0.9330289193302892, Precision 0.7349452601228124, Recall 0.7730923694779117, F1 0.7501576646300544



NameError: name 'all_predictions' is not defined

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')

plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')

plt.ylabel('Accuracy')

plt.xlabel('Epoch')

plt.legend()

plt.ylim([0, 1]);

In [ ]:
test_acc, _ = eval_model(

  model,

  test_data_loader,

  loss_fn,

  device,

  len(test_df)

)

test_acc.item()

In [ ]:
def get_predictions(model, data_loader):

  model = model.eval()

  text = []

  predictions = []

  real_values = []
  texts = []


  with torch.no_grad():

    for d in data_loader:


      input_ids = d["input_ids"].to(device)

      attention_mask = d["attention_mask"].to(device)
      token_type_ids = d["token_type_ids"].to(device)


      targets = d["label_"].to(device)

      outputs = model(

        input_ids=input_ids,

        attention_mask=attention_mask,
        token_type_ids = token_type_ids,

      )

      _, preds = torch.max(outputs, dim=1)

      #for xx in input_ids:
        #text.extend(tokenizer.decode(xx,skip_special_tokens=True))

      predictions.extend(preds)

      real_values.extend(d['label_'])
  texts.extend(text)

  predictions = torch.stack(predictions).cpu()

  #texts = torch.stack(texts).cpu()

  real_values = torch.stack(real_values).cpu()

  return  predictions, real_values

In [ ]:
y_pred, y_test = get_predictions(model,test_data_loader)


In [ ]:
text = []
for d in test_data_loader:
  for dd in d['input_ids']:
    text.append(tokenizer.decode(dd, skip_special_tokens=True))

In [ ]:

prediction = []
for i in y_pred:
  prediction.append(i.item())
len(prediction)

In [ ]:
real = []
for i in y_test:
  real.append(i.item())
len(real)

In [ ]:
test_result_df = pd.DataFrame()
test_result_df['text'] = text
test_result_df['prediction'] = prediction
test_result_df['real'] = real


In [ ]:
test_result_df.to_csv('test_result_prediction.tsv', sep = '|')

In [ ]:
#print(classification_report(y_test, y_pred, target_names=class_names))
#print(y_pred)
for i in y_pred:
  print(i)

In [ ]:
d = next(iter(test_data_loader))
print(d.keys())
test_df['label']
all_input = []
for i in test_data_loader:
  input = i['input_ids']
  all_input.extend(input)

all_input=torch.stack(all_input).cpu()

In [ ]:
all_input=torch.stack(all_input).cpu()

In [ ]:
predicted_label = [73,298,34,107,148,325,201,79,285,63,68,93,233,143,281,85,45,34,117,281,155,298,89,0,344,149]
pred_expansion = []
for i in predicted_label:
  for j, row in label_df.iterrows():
    if row[1] == i:
      pred_expansion. append(row[0])
pred_expansion